In [1]:
from os import listdir
from os.path import isfile, join
import cv2
import numpy as np
import sys
import os
import shutil

In [2]:
# path = "E:\\ML\\ComputerVision\\QiuMo\\dataset\\"
# file_names = [f for f in listdir(path) if isfile(join(path, f))]
# print(str(len(file_names)) + ' images loaded')

140 images loaded


In [3]:
# q_count = 0
# m_count = 0
training_size = 60
test_size = 10
training_images = []
training_labels = []
test_images = []
test_labels = []
size = 150

In [4]:
# q_dir_train = "E:\\ML\\ComputerVision\\QiuMo\\dataset\\train\\Qiuqiu\\"
# m_dir_train = "E:\\ML\\ComputerVision\\QiuMo\\dataset\\train\\Momo\\"
# q_dir_val = "E:\\ML\\ComputerVision\\QiuMo\\dataset\\test\\Qiuqiu\\"
# m_dir_val = "E:\\ML\\ComputerVision\\QiuMo\\dataset\\test\\Momo\\"

# def make_dir(directory):
#         if os.path.exists(directory):
#             shutil.rmtree(directory)
#         os.makedirs(directory)

# make_dir(q_dir_train)
# make_dir(m_dir_train)
# make_dir(q_dir_val)
# make_dir(m_dir_val)

In [5]:
# def getZeros(number):
#     if(number > 10 and number < 100):
#         return "0"
#     if(number < 10):
#         return "00"
#     else:
#         return ""
        
# for i, f in enumerate(file_names):
#     if file_names[i][0] == 'Q':
#         q_count += 1
#         img = cv2.imread(path+f)
#         img = cv2.resize(img, (size,size), interpolation=cv2.INTER_AREA)
#         if q_count <= training_size:
#             training_images.append(img)
#             training_labels.append(1)
#             cv2.imwrite(q_dir_train+'q_'+str(getZeros(q_count))+str(q_count)+'.jpg', img)
#         if q_count > training_size and q_count <= training_size + test_size:
#             test_images.append(img)
#             test_labels.append(1)
#             cv2.imwrite(q_dir_val+'q_'+str(getZeros(q_count-60))+str(q_count-60)+'.jpg', img)
    
#     if file_names[i][0] =='M':
#         m_count += 1
#         img = cv2.imread(path+f)
#         img = cv2.resize(img, (size,size), interpolation=cv2.INTER_AREA)
#         if m_count <= training_size:
#             training_images.append(img)
#             training_labels.append(0)
#             cv2.imwrite(m_dir_train+'m_'+str(getZeros(m_count))+str(m_count)+'.jpg', img)
#         if m_count > training_size and m_count <= training_size + test_size:
#             test_images.append(img)
#             test_labels.append(0)
#             cv2.imwrite(m_dir_val+'m_'+str(getZeros(m_count-60))+str(m_count-60)+'.jpg', img)
    
#     if q_count == training_size + test_size and m_count == training_size + test_size:
#         break


# print("Training and Test Data Extraction Complete")

Training and Test Data Extraction Complete


In [6]:
# print('Training data length: ', len(training_images))
# print('Test data length: ', len(test_images))

Training data length:  120
Test data length:  20


In [1]:
# np.savez('QM_training_data.npz', np.array(training_images))
# np.savez('QM_training_labels.npz', np.array(training_labels))
# np.savez('QM_test_data.npz', np.array(test_images))
# np.savez('QM_test_labels.npz', np.array(test_labels))

In [4]:
def load_data(datasetname):
    npzfile = np.load(datasetname + '_training_data.npz')
    train = npzfile['arr_0']

    npzfile = np.load(datasetname + '_training_labels.npz')
    train_labels = npzfile['arr_0']
    
    npzfile = np.load(datasetname + '_test_data.npz')
    test = npzfile['arr_0']
    
    npzfile = np.load(datasetname + '_test_labels.npz')
    test_labels = npzfile['arr_0']

    return (train, train_labels), (test, test_labels)

In [5]:
x_train = np.load('QM_training_data.npz')['arr_0']
y_train = np.load('QM_training_labels.npz')['arr_0']
x_test = np.load('QM_test_data.npz')['arr_0']
y_test = np.load('QM_test_labels.npz')['arr_0']


In [10]:
y_train = y_train.reshape(y_train.shape[0], 1)
y_test = y_test.reshape(y_test.shape[0],1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(120, 150, 150, 3)
(120, 1)
(20, 150, 150, 3)
(20, 1)


In [11]:
from __future__ import print_function
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Activation, Flatten
from tensorflow.keras.models import Sequential

In [14]:
batch_size = 5
epochs = 20

img_row = x_train[0].shape[0]
img_cols = x_train[1].shape[0]
input_shape = (img_row, img_cols, 3)

model = Sequential()
model.add(Conv2D(32, (3,3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='Adam',  metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
activation_10 (Activation)   (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_11 (Activation)   (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 34, 34, 64)       

In [15]:
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test), shuffle=True)

scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Train on 120 samples, validate on 20 samples
Epoch 1/20
120/120 [==============================] - 2s 15ms/sample - loss: 0.7196 - accuracy: 0.5167 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 2/20
120/120 [==============================] - 1s 12ms/sample - loss: 0.6933 - accuracy: 0.4833 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 3/20
120/120 [==============================] - 1s 12ms/sample - loss: 0.6904 - accuracy: 0.5000 - val_loss: 0.6934 - val_accuracy: 0.5000
Epoch 4/20
120/120 [==============================] - 1s 12ms/sample - loss: 0.7111 - accuracy: 0.4750 - val_loss: 0.6922 - val_accuracy: 0.6000
Epoch 5/20
120/120 [==============================] - 2s 13ms/sample - loss: 0.6864 - accuracy: 0.5750 - val_loss: 0.6570 - val_accuracy: 0.8000
Epoch 6/20
120/120 [==============================] - 1s 12ms/sample - loss: 0.6356 - accuracy: 0.5417 - val_loss: 0.6177 - val_accuracy: 0.7000
Epoch 7/20
120/120 [==============================] - 1s 12ms/sample - loss: 0.7575 -

In [16]:
classifier = model

def draw_test(name, pred, input_im):
    BLACK = [0,0,0]
    if pred == "[0]":
        pred = "Momo"
    if pred == "[1]":
        pred = "Qiuqiu"
    expanded_image = cv2.copyMakeBorder(input_im, 0, 0, 0, imageL.shape[0] ,cv2.BORDER_CONSTANT,value=BLACK)
    #expanded_image = cv2.cvtColor(expanded_image, cv2.COLOR_GRAY2BGR)
    cv2.putText(expanded_image, str(pred), (252, 70) , cv2.FONT_HERSHEY_COMPLEX_SMALL,4, (0,255,0), 2)
    cv2.imshow(name, expanded_image)


for i in range(0,5):
    rand = np.random.randint(0,len(x_test))
    input_im = x_test[rand]

    imageL = cv2.resize(input_im, None, fx=2, fy=2, interpolation = cv2.INTER_CUBIC)
    cv2.imshow("Test Image", imageL)

    input_im = input_im.reshape(1,150,150,3) 
    
    ## Get Prediction
    res = str(classifier.predict_classes(input_im, 1, verbose = 0)[0])

    draw_test("Prediction", res, imageL) 
    cv2.waitKey(0)

cv2.destroyAllWindows()

In [31]:
input_im = cv2.imread('./dataset/Validation/T12.jpg')
input_im = cv2.resize(input_im, (size,size), interpolation=cv2.INTER_AREA)
imageL = cv2.resize(input_im, None, fx=2, fy=2, interpolation = cv2.INTER_AREA)
cv2.imshow("Test Image", imageL)
input_im = input_im.reshape(1,150,150,3) 
res = str(classifier.predict_classes(input_im, 1, verbose = 0)[0])
draw_test("Prediction", res, imageL) 
cv2.waitKey(0)
cv2.destroyAllWindows()